In [1]:
from vessel_analysis_3d.processing_pipeline import Pipeline3D
from skimage.morphology import skeletonize
import numpy as np
from pathlib import Path
import pandas as pd
from aicsimageio import AICSImage
from skimage.morphology import label, remove_small_objects, remove_small_holes

In [2]:
from vessel_analysis_3d.graph.networkx_from_array import get_networkx_graph_from_array
from vessel_analysis_3d.graph.core import GraphObj
from vessel_analysis_3d.graph.stats_reporting import report_everything

In [3]:
# make sure setting the pixel Dimension properly. For other parameters, please refer
# https://www.cell.com/cell-reports-methods/pdf/S2667-2375(23)00051-6.pdf
# see Table 1 on page 4.
params = {
    "pixelDimensions": [1.0, 1.0, 1.0],
    "pruningScale": 1,
    "lengthLimit": 1,
    "diaScale": 1,
    "branchingThreshold": 0.25
}

In [7]:
pred_path = Path("/path/to/segmentation/results")
filenames = sorted(pred_path.glob("*.tiff"))

# in this analysis, we will generate one or two "raw" statistics for each file (if valid string
# vessels are detected, then there will be two statistics: one for all vessels and one for only
# string vessels. If no valid string vessels are detected, then only one statistics for all 
# vessels), and also a summary csv. Here, we specify the path to save the "raw" statistics 
# and the filename of the summary file
out_path = Path("/path/to/save/all/stats")
save_name = out_path / Path("data_full_stats.csv")

In [ ]:
pred = AICSImage(filenames[0]).get_image_data("ZYX", C=0, T=0)
string_vessel = pred == 2
all_vessel = pred > 0

string_skl = skeletonize(string_vessel > 0, method="lee").astype(np.uint8)
string_skl[string_skl > 0] = 1

skl_final, brPts, endPts, reports = Pipeline3D.process_one_file(string_vessel, string_skl, params)

stats = []
num = 0
for fn in filenames:
    num = num + 1
    print("--Analyzing the result: ", num,'/',len(filenames), "...")

    fileID = fn.stem
    pred = AICSImage(fn).get_image_data("ZYX", C=0, T=0)

    # here, in this demo, we run analysis on string vessel only and on all vessels
    # we can also run on normal vessel only (pred == 1)
    string_vessel = pred == 2
    all_vessel = pred > 0

    _, num_string = label(string_vessel, connectivity=3, return_num=True)

    all_skl = skeletonize(all_vessel > 0, method="lee").astype(np.uint8)
    all_skl[all_skl > 0] = 1
    _, _, _, all_reports = Pipeline3D.process_one_file(all_vessel, all_skl, params)

    raw_stats_name = f"{fileID}_all_vessels_stats.csv"
    all_reports[0].to_csv(raw_stats_name, index=False)

    if num_string > 0:

        string_skl = skeletonize(string_vessel > 0, method="lee").astype(np.uint8)
        string_skl[string_skl > 0] = 1

        # skeleton to graph
        networkxGraph = get_networkx_graph_from_array(string_skl)

        # Statistical Analysis
        gh = GraphObj(string_vessel, string_skl, networkxGraph, **params)
        skl_final = gh.prune_and_analyze(return_final_skel=True)

        if np.count_nonzero(skl_final) < 3:
            stats.append(
                {
                    "filename": fileID,
                    "num_string_vessel": 0,
                    "average_thickness_string": 0,
                    "average_straightness_string": 0,
                    "average_length_string": 0,
                    "string_to_all_ratio": 0,
                    "average_thickness_all": np.mean(all_reports[0]["diameter"]),
                    "average_straightness_all": np.mean(all_reports[0]["straightness"]),
                    "average_length_all": np.mean(all_reports[0]["length"]),
                }
            )
        else:
            string_reports = report_everything(gh, "default")
            # save the string report
            raw_string_stats_name = f"{fileID}_string_vessels_stats.csv"
            string_reports[0].to_csv(raw_string_stats_name, index=False)
            stats.append(
                {
                    "filename": fileID,
                    "num_string_vessel": num_string,
                    "average_thickness_string": np.mean(string_reports[0]["diameter"]),
                    "average_straightness_string": np.mean(string_reports[0]["straightness"]),
                    "average_length_string": np.mean(string_reports[0]["length"]),
                    "string_to_all_ratio": len(string_reports[0]) / len(all_reports[0]),
                    "average_thickness_all": np.mean(all_reports[0]["diameter"]),
                    "average_straightness_all": np.mean(all_reports[0]["straightness"]),
                    "average_length_all": np.mean(all_reports[0]["length"]),
                }
            )
    else:
        stats.append(
            {
                "filename": fileID,
                "num_string_vessel": 0,
                "average_thickness_string": 0,
                "average_straightness_string": 0,
                "average_length_string": 0,
                "string_to_all_ratio": 0,
                "average_thickness_all": np.mean(all_reports[0]["diameter"]),
                "average_straightness_all": np.mean(all_reports[0]["straightness"]),
                "average_length_all": np.mean(all_reports[0]["length"]),
            }
        )

stats_df = pd.DataFrame(stats)
# save_name = "Data_1_full_stats.csv"
stats_df.to_csv(save_name, index=False)